In [1]:
from math import radians, sin, sqrt

import cadquery as cq
import ipywidgets as widgets
import jupyter_cadquery as jcq
from jupyter_cadquery.replay import replay, show

Overwriting auto display for cadquery Workplane and Shape


In [16]:
jcq.set_defaults(
    control="orbit",
    axes=True,
    axes0=True,
    grid=(True, False, False),
    glass=True,
    theme="browser",
)

# open_viewer("Cadquery")
replay_box = widgets.Checkbox(False, description="Enable Replay")
out = widgets.Output()
display(replay_box)
display(out)


@out.capture()
def changed(e):
    if e["new"]:
        jcq.replay.enable_replay(warning=False)
    else:
        jcq.replay.disable_replay()


replay_box.observe(changed, names="value")

Checkbox(value=False, description='Enable Replay')

Output()

In [8]:
teeth_pts = [
    (0.747183, 0),
    (0.647876, 0.037218),
    (0.598311, 0.130528),
    (0.578556, 0.238423),
    (0.547158, 0.343077),
    (0.504649, 0.443762),
    (0.451556, 0.53975),
    (0.358229, 0.636924),
    (0.2484, 0.707276),
    (0.127259, 0.750044),
    (0, 0.76447),
    (-0.127259, 0.750044),
    (-0.2484, 0.707276),
    (-0.358229, 0.636924),
    (-0.451556, 0.53975),
    (-0.504797, 0.443762),
    (-0.547291, 0.343077),
    (-0.578605, 0.238423),
    (-0.598311, 0.130528),
    (-0.648009, 0.037218),
    (-0.747183, 0),
]

one_teeth = (
    cq.Workplane("XY")
    .moveTo(teeth_pts[0][0], -0.5)
    .lineTo(teeth_pts[0][0], teeth_pts[0][1])
    .spline(teeth_pts)
    .lineTo(-teeth_pts[0][0], -0.5)
    .close()
)

replay(one_teeth.extrude(-1).polyline(teeth_pts))

Use the multi select box below to select one or more steps you want to examine


In [9]:
teeth = cq.importers.importDXF("gt2_27teeth.dxf")

replay(teeth)

Use the multi select box below to select one or more steps you want to examine


In [10]:
flange = 19
flange_thickness = 1.8
r = (
    cq.Workplane("XY")
    .add(teeth)
    .wires()
    .toPending()
    .extrude(6)
    .faces(">Z")
    .circle(flange / 2)
    .extrude(flange_thickness)
    .faces(">Z[1]")
    .edges("not %Line")
    .chamfer(0.5)
    .faces("<Z")
    .workplane()
    .cboreHole(9, 10, 4 - flange_thickness - 0.2)
    .faces(">Z")
    .workplane()
    .polygon(3, 10 + 2, forConstruction=True)
    .vertices()
    .hole(2 - 0.2)
)

r2 = (
    cq.Workplane("XY")
    .circle(flange / 2)
    .extrude(1.5)
    .faces(">Z")
    .edges()
    .chamfer(0.5)
    .faces(">Z")
    .hole(10)
    .faces(">Z")
    .workplane()
    .polygon(3, 10 + 2, forConstruction=True)
    .vertices()
    .hole(2)
)

replay(r)
replay(r2)

Use the multi select box below to select one or more steps you want to examine


Use the multi select box below to select one or more steps you want to examine


In [12]:
mount = (
    cq.Workplane("XY")
    .box(50, 20, 4)
    .faces(">Z")
    .circle((5 + 2) / 2)
    .extrude(2)
    .faces(">Z")
    .workplane()
    .hole(4.5)
    .faces("<Z")
    .workplane()
    .pushPoints([(-20, 0), (20, 0)])
    .hole(5)
)

replay(mount)
cq.exporters.export(mount, "/tmp/x.stl")

Use the multi select box below to select one or more steps you want to examine


In [7]:
cq.exporters.export(r.add(r2.translate([20, 0])), "/tmp/x.stl")